In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [6]:
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:100000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [7]:
df.isnull().sum()

Sentence #    95456
Word              0
POS               0
Tag               0
dtype: int64

In [8]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(4544, 10922, 17)

In [9]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,75
1,B-eve,53
2,B-geo,3303
3,B-gpe,1740
4,B-nat,30
5,B-org,1876
6,B-per,1668
7,B-tim,1823
8,I-art,43
9,I-eve,47


In [10]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((67000, 15507), (67000,))

In [11]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [12]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [13]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [14]:
X_train[:10]

[[{'bias': 1.0,
   'word.lower()': 'a',
   'word[-3:]': 'A',
   'word[-2:]': 'A',
   'word.isupper()': True,
   'word.istitle()': True,
   'word.isdigit()': False,
   'postag': 'DT',
   'postag[:2]': 'DT',
   'BOS': True,
   '+1:word.lower()': 'second',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'JJ',
   '+1:postag[:2]': 'JJ'},
  {'bias': 1.0,
   'word.lower()': 'second',
   'word[-3:]': 'ond',
   'word[-2:]': 'nd',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'JJ',
   'postag[:2]': 'JJ',
   '-1:word.lower()': 'a',
   '-1:word.istitle()': True,
   '-1:word.isupper()': True,
   '-1:postag': 'DT',
   '-1:postag[:2]': 'DT',
   '+1:word.lower()': 'nominee',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'NN',
   '+1:postag[:2]': 'NN'},
  {'bias': 1.0,
   'word.lower()': 'nominee',
   'word[-3:]': 'nee',
   'word[-2:]': 'ee',
   'word.isupper()': False,
   'word.istitle()': 

In [15]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [17]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-art       1.00      0.03      0.07        29
       B-eve       0.86      0.25      0.39        24
       B-geo       0.75      0.88      0.81      1043
       B-gpe       0.89      0.78      0.83       588
       B-nat       0.67      0.20      0.31        10
       B-org       0.75      0.64      0.69       649
       B-per       0.81      0.81      0.81       546
       B-tim       0.90      0.85      0.87       589
       I-art       0.00      0.00      0.00         7
       I-eve       0.57      0.22      0.32        18
       I-geo       0.71      0.71      0.71       204
       I-gpe       0.47      0.53      0.50        17
       I-nat       1.00      0.50      0.67         2
       I-org       0.78      0.73      0.76       545
       I-per       0.80      0.90      0.85       574
       I-tim       0.79      0.68      0.73       185
           O       0.99      0.99      0.99     28019

   micro avg       0.96   

/Users/machnev/anaconda3/envs/torch_env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-art  -> I-art   5.467085
I-art  -> I-art   5.437765
B-eve  -> I-eve   5.415001
B-geo  -> I-geo   5.226918
I-tim  -> I-tim   5.094604
B-per  -> I-per   5.042157
B-gpe  -> I-gpe   4.985853
I-gpe  -> I-gpe   4.877554
I-eve  -> I-eve   4.837924
B-tim  -> I-tim   4.802178
B-org  -> I-org   4.475706
I-geo  -> I-geo   4.457340
I-org  -> I-org   4.299293
I-per  -> I-per   3.897607
B-nat  -> I-nat   3.693849
O      -> O       3.560786
I-nat  -> I-nat   2.449510
B-org  -> B-art   2.334343
B-geo  -> B-tim   1.647575
O      -> B-per   1.640416

Top unlikely transitions:
O      -> I-eve   -1.847486
B-geo  -> I-per   -1.872669
B-gpe  -> I-org   -1.945107
I-per  -> I-org   -1.967871
B-geo  -> I-org   -1.993363
B-org  -> I-geo   -2.027038
I-org  -> B-org   -2.271059
B-org  -> B-org   -2.284660
B-org  -> I-per   -2.309206
B-gpe  -> I-geo   -2.327449
I-org  -> I-per   -2.332108
O      -> I-art   -2.343913
B-tim  -> B-tim   -2.517921
O      -> I-per   -3.017004
B-gpe  -> B-gpe  

In [18]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
5.265235 B-tim    word[-3:]:day
4.720279 O        BOS
4.235628 O        bias
4.045893 O        word.lower():jewish
3.776394 B-per    word.lower():president
3.639942 I-tim    word[-3:]:day
3.602543 B-org    word.lower():al-qaida
3.573775 B-tim    word.lower():thanksgiving
3.495362 B-tim    word[-2:]:ay
3.494121 O        word.lower():kurdish
3.435182 O        word[-2:]:N1
3.364693 B-tim    word.lower():afternoon
3.350323 B-tim    word[-3:]:ber
3.339704 B-org    word.lower():hamas
3.270049 B-org    word.lower():parliament
3.241910 B-tim    word[-2:]:0s
3.190436 B-tim    +1:word.lower():year
3.176004 B-gpe    word.lower():nepal
3.158097 B-per    word.lower():prime
3.154870 B-gpe    word[-3:]:pal
3.144851 B-per    word.lower():gotovina
3.117080 B-per    word.lower():obama
3.092026 B-geo    -1:word.lower():serb
3.084035 B-geo    word.lower():mid-september
3.045427 I-geo    +1:word.lower():town
3.040978 B-org    -1:word.lower():telephoned
3.030600 O        +1:word.lower():minist